In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import getpass
import os
import pandas as pd
import numpy as np
import wikipedia
from nltk.tokenize import sent_tokenize
import tensorflow_hub as hub
import spacy
import nltk
from scipy import spatial

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_20321/1660279103.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-15 20:36:03.922085: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", 
google_api_key="removed", safety_settings=safety_settings)

# Evidence retrieval with Wikipedia

In [3]:
def wiki_search(text):
    try:
       page = wikipedia.page(text, auto_suggest=False)
    except wikipedia.DisambiguationError as e:
        sel = e.options[0]
        print(e.options)
        page = wikipedia.page(sel)
        
    return page.content

In [28]:
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [21]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


False

In [18]:
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
import torch
from transformers import BertTokenizer, BertModel, pipeline

def text_embedding(data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
    
    def get_bert_embeddings(data):
        tokens = tokenizer(data.tolist(), padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            embeddings = bert_model(**tokens).last_hidden_state.mean(dim=1)
        return embeddings

In [12]:
def get_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    return entities
    
def retrieve_documents(claim):
    docs = []
    entities = get_entities(claim)
    
    for entity in entities:
        print(entity)
        docs.append(wiki_search(entity))
    return docs

def compute_similarity(sentence1, sentence2):
    
    embedding1 = text_embedding([sentence1])[0].numpy()
    embedding2 = text_embedding([sentence2])[0].numpy()

    # Compute the cosine similarity between the two sentence embeddings
    similarity = 1 - spatial.distance.cosine(embedding1, embedding2)

    return similarity

def top_k_sentences(claim, split_sentences, k):
    sims = []
    
    for sentence in split_sentences: 
        sims.append((sentence, compute_similarity(sentence, claim)))

        #also post to db?

    sims = sorted(sims, key=lambda x: x[1], reverse = True)
    return [elem[0] for elem in sims[:k]]

def get_wiki_evidence(claim, k):
    docs = retrieve_documents(claim)
    evidence = []
    
    for doc in docs:
        sentences = sent_tokenize(doc)
        evidence += top_k_sentences(claim, sentences, k)
        
    return evidence


In [10]:
from scipy import spatial

In [19]:
get_wiki_evidence('Joe Biden created COVID', 3)

Joe Biden
COVID


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/orenciolli/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.9/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.9/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.9/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


# LLM

In [41]:
def evaluate_claim(claim):

    #articles = get_article_evidence(claim)
    ground_truth = get_wiki_evidence(claim)


    prompt = f"""please rate the veracity of the following claim on a scale from 0 to 5,
    with 0 being completely true and 5 being entirely false.
    Please ensure that the first character in your response is a single integer between 0 and 5,
    and explain your reasoning:
    {claim}"""

    response = llm.invoke(prompt).content

    rating = response[0]

    return response, int(rating)

## LIAR evaluation

In [6]:
df = pd.read_csv('liar_plus/test2.tsv', delimiter='\t', header = None)

df = df.drop(columns = [0])
df.rename({1: 'id', 2: 'label', 3: 'statement', 4: 'subject', 5: 'speaker', 6: 'job-title',
           7: 'state_info', 8: 'party_affiliation', 9: 'barely_true_counts', 10: 'false_counts',
           11: 'half_true_counts', 12: 'mostly_true_counts', 13: 'pants_on_fire_counts', 14: 'context',
           15: 'justification'
          }, axis = 1, inplace = True)

In [23]:
label_map = {'pants-fire': 5, 'false': 4, 'half-true': 3, 
             'barely-true': 2, 'mostly-true': 1, 'true': 0}

df['label'] = df['label'].replace(label_map)

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_93212/398931798.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace(label_map)


In [15]:
#preds = df['statement'].apply(lambda x: evaluate_claim(x)[1] )

In [27]:
preds = []
labels = []

for index, row in df.iterrows():
    try:
        result = evaluate_claim(row['statement'])[1]

        preds.append(result)
        labels.append(row['label'])
    except:
        continue

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


In [42]:
results_df = pd.DataFrame({'predicted': preds, 'label': labels})

results_df['predicted'] = results_df['predicted'].astype(int)
(results_df['predicted'] == results_df['label']).mean()

0.17857142857142858

In [45]:
df['statement'].iloc[0]

'Building a wall on the U.S.-Mexico border will take literally years.'

In [44]:
evaluate_claim(df['statement'].iloc[0])

('0\nBuilding a wall on the U.S.-Mexico border is a complex and time-consuming project that would take years to complete. According to experts, the construction of a wall along the entire U.S.-Mexico border would take at least 3.5 years and cost billions of dollars.',
 0)

In [40]:
results_df.dtypes

predicted    object
label         int64
dtype: object

In [51]:
(abs(results_df['predicted'].astype(int) - results_df['label'].astype(int)) <= 1).mean()

0.5124584717607974